In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import os
from pathlib import Path

import numpy as np

from compute_f import split_ts_seq, compute_step_positions
from io_f import read_data_file
from visualize_f import visualize_trajectory, visualize_heatmap, save_figure_to_html

floor_data_dir = './data/site1/F1'
path_data_dir = floor_data_dir + '/path_data_files'
floor_plan_filename = floor_data_dir + '/floor_image.png'
floor_info_filename = floor_data_dir + '/floor_info.json'

save_dir = './output/site1/F1'
path_image_save_dir = save_dir + '/path_images'
step_position_image_save_dir = save_dir
magn_image_save_dir = save_dir
wifi_image_save_dir = save_dir + '/wifi_images'
ibeacon_image_save_dir = save_dir + '/ibeacon_images'
wifi_count_image_save_dir = save_dir

In [3]:
Path(path_image_save_dir).mkdir(parents=True, exist_ok=True)
Path(magn_image_save_dir).mkdir(parents=True, exist_ok=True)
Path(wifi_image_save_dir).mkdir(parents=True, exist_ok=True)
Path(ibeacon_image_save_dir).mkdir(parents=True, exist_ok=True)

with open(floor_info_filename) as f:
    floor_info = json.load(f)
width_meter = floor_info["map_info"]["width"]
height_meter = floor_info["map_info"]["height"]

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

# Investigating a trace file

In [ ]:
from tqdm.auto import tqdm
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:
path_filenames = list(Path(path_data_dir).resolve().glob("*.txt"))
path_filenames = path_filenames[:10]

In [ ]:
# Collect ground truth trajectories (path_id, [(time, x, y)])
gts = []
for path_filename in tqdm(path_filenames):
    path_data = read_data_file(path_filename)
    path_id = path_filename.name.split(".")[0]
    gts.append((path_id, path_data.waypoint))

In [ ]:
# Building figs
figs = []
for path_id, waypoint in tqdm(gts):
    figs.append(visualize_trajectory(waypoint[:, 1:3], floor_plan_filename, width_meter, height_meter, title=path_id, show=False, plot_width=700))

In [ ]:
# Computing stats
# ToDo: use integrations/helper.py > to_plt to convert data to Geolife plt file to use all Stats utils
stats = []
for _, waypoint in tqdm(gts):
    # Collect nb of points, times between waypoints, speeds, ...
    nb_points = len(waypoint)
    pass

In [ ]:
@interact(
    i=widgets.IntSlider(min=0, max=len(figs) - 1, step=1, value=5)
)
def show_path(i):
    figs[i].show()